<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=5c9e3ad359bab01bc834605bb3e47bfbcea7d1881363cf09cb98b276faf1258c
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-17 09:34:15
-------------------
qualified stocks: 87
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  3.00 K
Current:  1.46 C
-------------------
Today PnL: 59.85 K (0.41%)
Current PnL: -19.63 L (-12.87%)
CY Booked + Current PnL: -6.11 L (-4.01%)
-------------------
Total profit:  2.13 L
Total loss:  -21.76 L
-------------------
Total Booked + Current PnL: 20.87 L (16.61%)
Total Booked PnL: 40.50 L (32.24%)
Curr Year Booked PnL: 13.52 L (9.27%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.36 C
Est FTT PnL: 90.51 L (62.08%)
Deployed:  1.26 C
Current:  1.46 C
CAGR/XIRR %: 8.61%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,147.77,84.0,M-SC,0.70,162335.0,-12224.0,12256.0,0.80,-7.00,7.55,0.02,234.0,-1.00,1.11,41.20,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,101.92,51.0,M-SC,3.42,86971.0,-13806.0,13898.0,0.05,-13.70,15.98,0.09,253.0,-0.99,0.60,14.26,OX40N,NTT,DURABLES
2,ABBOTINDIA,35195.00,-15.75,43.0,X-MC,3.59,87975.0,-2703.0,17613.0,-0.05,-2.98,20.02,16.44,63.0,-0.15,0.60,16.59,X40,ATH,PHARMA
43,ITC,452.00,-40.34,51.0,X-LC,0.93,200294.0,156.0,20730.0,0.36,0.08,10.35,10.44,5.0,0.01,1.37,5.65,X40,NTT,FMCG
35,ICICIGI,2252.93,-15.31,57.0,X-MC,7.49,188334.0,14014.0,21188.0,1.00,8.04,11.25,20.19,68.0,0.66,1.29,24.37,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RAJESHEXPO,518.0,1787.63,68.0,L-SC,2.75,51503.0,-85674.0,85768.0,3.75,-62.46,166.53,0.07,268.0,-1.00,0.35,28.09,OX40N,NTT,JEWELLERY
22,DEN,75.0,86.53,44.0,M-SC,5.95,52224.0,-32240.0,67776.0,2.67,-38.17,129.78,42.07,235.0,-0.48,0.36,10.91,AR,NTT,ENTERTAINMENT
75,TITAGARH,1548.0,11.21,52.0,H-SC,4.48,202054.0,-40123.0,150894.0,2.28,-16.57,74.68,45.74,174.0,-0.27,1.39,31.42,XY24,NTT,ENGINEERING
60,REPCOHOME,880.0,-54.55,55.0,H-SC,10.67,262186.0,-24319.0,281640.0,2.18,-8.49,107.42,89.82,160.0,-0.09,1.80,38.72,XY24,NTT,FINANCE
70,SURYODAY,240.0,55.70,46.0,H-SC,13.38,143259.0,-35812.0,101542.0,2.12,-20.00,70.88,36.71,167.0,-0.35,0.98,42.14,XR,NTT,BANKS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,1065.0,-20.63,26.0,X-LC,0.28,244512.0,-31898.0,453056.0,-4.58,-11.54,185.29,152.37,3.0,-0.07,1.68,0.00,XY24,NTT,AUTO
19,COFFEEDAY,80.0,-51.71,32.0,L-SC,21.18,71232.0,-42317.0,82366.0,-3.39,-37.27,115.63,35.27,270.0,-0.51,0.49,74.34,XR,NTT,HOTELS
24,EASEMYTRIP,26.4,-21.71,33.0,M-SC,20.26,75429.0,-101386.0,182184.0,-2.77,-57.34,241.53,45.70,205.0,-0.56,0.52,0.00,XY24,NTT,TRAVEL
12,BATAINDIA,2096.0,11.05,25.0,X-SC,9.75,84735.0,-43935.0,89234.0,-0.60,-34.15,105.31,35.20,92.0,-0.49,0.58,0.00,X40,NTT,FOOTWEAR
14,BSOFT,831.7,-9.93,61.0,H-SC,8.60,98558.0,-42095.0,113529.0,-0.57,-29.93,115.19,50.79,151.0,-0.37,0.68,14.75,XR,ATH,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WHIRLPOOL,2270.00,-41.83,39.0,M-SC,8.89,114025.0,-773.0,94812.0,-0.45,-0.67,83.15,81.92,236.0,-0.01,0.78,34.99,XR,NTT,DURABLES
25,FINCABLES,1641.55,-18.12,51.0,M-SC,10.18,148889.0,1245.0,158075.0,1.68,0.84,106.17,107.91,216.0,0.01,1.02,3.87,OX40N,ATH,CABLES
86,ZYDUSLIFE,1286.17,-14.97,36.0,H-MC,5.60,207939.0,-1635.0,76314.0,-0.38,-0.78,36.70,35.63,119.0,-0.02,1.43,15.76,AR,ATH,PHARMA
85,WIPRO,420.00,-13.79,53.0,M-LC,5.57,151633.0,688.0,108766.0,0.08,0.46,71.73,72.51,99.0,0.01,1.04,6.51,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-18.12,51.0,M-SC,10.18,148889.0,1245.0,158075.0,1.68,0.84,106.17,107.91,216.0,0.01,1.02,3.87,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,147.77,84.0,M-SC,0.70,162335.0,-12224.0,12256.0,0.80,-7.00,7.55,0.02,234.0,-1.00,1.11,41.20,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,101.92,51.0,M-SC,3.42,86971.0,-13806.0,13898.0,0.05,-13.70,15.98,0.09,253.0,-0.99,0.60,14.26,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-65.98,50.0,H-SC,2.37,223371.0,-46296.0,82625.0,-0.25,-17.17,36.99,13.47,158.0,-0.56,1.53,13.58,XY24,NTT,PAINTS
67,SIS,528.00,2026.94,48.0,H-SC,2.35,85846.0,-25186.0,48795.0,0.13,-22.68,56.84,21.26,166.0,-0.52,0.59,15.99,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,STARHEALTH,761.00,40.12,75.0,H-SC,15.63,288837.0,7714.0,125153.0,1.63,2.74,43.33,47.26,171.0,0.06,1.98,54.75,XY24,NTT,INSURANCE
62,SAMMAANCAP,326.00,18.70,54.0,M-SC,36.85,159660.0,10440.0,133747.0,0.72,7.00,83.77,96.62,245.0,0.08,1.10,72.79,XY25,NTT,FINANCE
85,WIPRO,420.00,-13.79,53.0,M-LC,5.57,151633.0,688.0,108766.0,0.08,0.46,71.73,72.51,99.0,0.01,1.04,6.51,XR,NTT,IT
25,FINCABLES,1641.55,-18.12,51.0,M-SC,10.18,148889.0,1245.0,158075.0,1.68,0.84,106.17,107.91,216.0,0.01,1.02,3.87,OX40N,ATH,CABLES
38,INDIAMART,4810.62,-47.96,49.0,H-SC,4.41,129394.0,6058.0,125564.0,-0.41,4.91,97.04,106.72,139.0,0.05,0.89,29.44,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-47.96,49.0,H-SC,4.41,129394.0,6058.0,125564.0,-0.41,4.91,97.04,106.72,139.0,0.05,0.89,29.44,AR,ATH,MISC
25,FINCABLES,1641.55,-18.12,51.0,M-SC,10.18,148889.0,1245.0,158075.0,1.68,0.84,106.17,107.91,216.0,0.01,1.02,3.87,OX40N,ATH,CABLES
85,WIPRO,420.00,-13.79,53.0,M-LC,5.57,151633.0,688.0,108766.0,0.08,0.46,71.73,72.51,99.0,0.01,1.04,6.51,XR,NTT,IT
84,WHIRLPOOL,2270.00,-41.83,39.0,M-SC,8.89,114025.0,-773.0,94812.0,-0.45,-0.67,83.15,81.92,236.0,-0.01,0.78,34.99,XR,NTT,DURABLES
86,ZYDUSLIFE,1286.17,-14.97,36.0,H-MC,5.60,207939.0,-1635.0,76314.0,-0.38,-0.78,36.70,35.63,119.0,-0.02,1.43,15.76,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,11.05,25.0,X-SC,9.75,84735.0,-43935.0,89234.0,-0.60,-34.15,105.31,35.20,92.0,-0.49,0.58,0.00,X40,NTT,FOOTWEAR
76,TMPV,1065.00,-20.63,26.0,X-LC,0.28,244512.0,-31898.0,453056.0,-4.58,-11.54,185.29,152.37,3.0,-0.07,1.68,0.00,XY24,NTT,AUTO
55,QUESS,986.00,-53.17,28.0,X-SC,37.27,46279.0,-18727.0,169654.0,-0.25,-28.81,366.59,232.18,83.0,-0.11,0.32,0.00,XY24,NTT,MISC
23,DMART,5391.45,-17.02,32.0,X-LC,5.28,202455.0,-7952.0,67114.0,-0.11,-3.78,33.15,28.12,19.0,-0.12,1.39,18.95,X40N,ATH,FMCG
27,GLAXO,3466.20,-19.11,33.0,X-MC,6.37,200712.0,1428.0,76592.0,1.19,0.72,38.16,39.15,60.0,0.02,1.38,29.93,X40,ATH,PHARMA


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,1065.00,-20.63,26.0,X-LC,0.28,244512.0,-31898.0,453056.0,-4.58,-11.54,185.29,152.37,3.0,-0.07,1.68,0.00,XY24,NTT,AUTO
43,ITC,452.00,-40.34,51.0,X-LC,0.93,200294.0,156.0,20730.0,0.36,0.08,10.35,10.44,5.0,0.01,1.37,5.65,X40,NTT,FMCG
34,HONAUT,58357.33,-25.10,36.0,X-SC,0.99,106530.0,-21408.0,68541.0,-0.06,-16.73,64.34,36.84,90.0,-0.31,0.73,9.06,X40N,ATH,ELECTRICAL
36,ICICIPRULI,790.00,-20.00,67.0,X-MC,1.19,184927.0,8189.0,46546.0,0.27,4.63,25.17,30.97,75.0,0.18,1.27,17.78,X40,ATH,INSURANCE
66,SIEMENS,4671.50,-3.51,55.0,H-LC,1.59,157350.0,-28745.0,76220.0,2.04,-15.45,48.44,25.51,49.0,-0.38,1.08,15.95,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,11.05,25.0,X-SC,9.75,84735.0,-43935.0,89234.0,-0.60,-34.15,105.31,35.20,92.0,-0.49,0.58,0.00,X40,NTT,FOOTWEAR
55,QUESS,986.00,-53.17,28.0,X-SC,37.27,46279.0,-18727.0,169654.0,-0.25,-28.81,366.59,232.18,83.0,-0.11,0.32,0.00,XY24,NTT,MISC
76,TMPV,1065.00,-20.63,26.0,X-LC,0.28,244512.0,-31898.0,453056.0,-4.58,-11.54,185.29,152.37,3.0,-0.07,1.68,0.00,XY24,NTT,AUTO
53,PAGEIND,51769.93,-27.72,43.0,X-MC,7.71,160620.0,-3120.0,46467.0,0.98,-1.91,28.93,26.47,55.0,-0.07,1.10,1.76,X40,ATH,APPARELS
10,BAJAJHFL,181.50,-18.15,38.0,X-MC,6.61,176677.0,-23829.0,120988.0,0.18,-11.88,68.48,48.45,64.0,-0.20,1.21,1.88,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,986.00,-53.17,28.0,X-SC,37.27,46279.0,-18727.0,169654.0,-0.25,-28.81,366.59,232.18,83.0,-0.11,0.32,0.00,XY24,NTT,MISC
59,RELAXO,1176.00,-41.24,39.0,X-SC,3.24,80107.0,-65053.0,140980.0,-0.01,-44.81,175.99,52.31,91.0,-0.46,0.55,7.23,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,11.05,25.0,X-SC,9.75,84735.0,-43935.0,89234.0,-0.60,-34.15,105.31,35.20,92.0,-0.49,0.58,0.00,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-15.75,43.0,X-MC,3.59,87975.0,-2703.0,17613.0,-0.05,-2.98,20.02,16.44,63.0,-0.15,0.60,16.59,X40,ATH,PHARMA
34,HONAUT,58357.33,-25.10,36.0,X-SC,0.99,106530.0,-21408.0,68541.0,-0.06,-16.73,64.34,36.84,90.0,-0.31,0.73,9.06,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.96,-26.54,59.0,X-LC,8.48,291522.0,-54434.0,121127.0,-0.15,-15.73,41.55,19.28,1.0,-0.45,2.00,7.77,X40,ATH,IT
41,INFY,2275.00,-20.32,52.0,X-LC,4.01,319606.0,6718.0,164981.0,-0.15,2.15,51.62,54.87,2.0,0.04,2.19,10.61,X40,BTT,IT
76,TMPV,1065.00,-20.63,26.0,X-LC,0.28,244512.0,-31898.0,453056.0,-4.58,-11.54,185.29,152.37,3.0,-0.07,1.68,0.00,XY24,NTT,AUTO
82,VBL,671.64,-18.11,48.0,X-LC,5.99,293416.0,-22426.0,135089.0,0.11,-7.10,46.04,35.67,4.0,-0.17,2.01,5.77,X40N,ATH,FMCG
43,ITC,452.00,-40.34,51.0,X-LC,0.93,200294.0,156.0,20730.0,0.36,0.08,10.35,10.44,5.0,0.01,1.37,5.65,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-51.71,32.0,L-SC,21.18,71232.0,-42317.0,82366.0,-3.39,-37.27,115.63,35.27,270.0,-0.51,0.49,74.34,XR,NTT,HOTELS
7,ASIANTILES,137.00,7627.78,65.0,L-SC,17.14,84791.0,-9019.0,85639.0,-0.00,-9.61,101.00,81.67,271.0,-0.11,0.58,64.20,XR,NTT,CERAMICS
51,MASFIN,398.61,-17.34,55.0,H-SC,7.50,93840.0,-4140.0,25740.0,1.15,-4.23,27.43,22.05,164.0,-0.16,0.64,36.40,XR,ATH,FINANCE
70,SURYODAY,240.00,55.70,46.0,H-SC,13.38,143259.0,-35812.0,101542.0,2.12,-20.00,70.88,36.71,167.0,-0.35,0.98,42.14,XR,NTT,BANKS
80,VAIBHAVGBL,521.00,73.55,62.0,H-SC,9.47,150391.0,-32384.0,143458.0,0.58,-17.72,95.39,60.77,159.0,-0.23,1.03,35.44,XR,NTT,JEWELLERY


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,147.77,84.0,M-SC,0.70,162335.0,-12224.0,12256.0,0.80,-7.00,7.55,0.02,234.0,-1.00,1.11,41.20,OX40N,NTT,PAINTS
80,VAIBHAVGBL,521.00,73.55,62.0,H-SC,9.47,150391.0,-32384.0,143458.0,0.58,-17.72,95.39,60.77,159.0,-0.23,1.03,35.44,XR,NTT,JEWELLERY
6,ASIANPAINT,3465.66,0.91,87.0,X-LC,9.56,256626.0,4858.0,48348.0,0.34,1.93,18.84,21.13,31.0,0.10,1.76,38.04,X40,ATH,PAINTS
14,BSOFT,831.70,-9.93,61.0,H-SC,8.60,98558.0,-42095.0,113529.0,-0.57,-29.93,115.19,50.79,151.0,-0.37,0.68,14.75,XR,ATH,IT
40,INDUSINDBK,1800.00,-698.49,70.0,L-MC,6.17,48447.0,-28760.0,54154.0,0.20,-37.25,111.78,32.89,259.0,-0.53,0.33,33.42,XR,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.60
1,25,44.43
2,50,75.73


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    47.62
MC    27.74
LC    24.60
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.44
X40      20.48
X40N     12.23
XY25     10.10
XR        9.55
AR        8.99
OX40N     7.54
X200      1.80
SR        1.00
MH        0.83
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.30
X-MC    21.38
X-LC    19.69
M-SC    13.35
X-SC     8.55
H-MC     4.66
H-LC     2.88
L-SC     1.42
M-MC     1.37
M-LC     1.04
L-LC     0.99
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.63,-3.75,37.30
IT,12.44,-15.87,77.82
FINANCE,11.94,-8.61,57.84
MISC,7.54,-18.17,77.78
PAINTS,6.01,-5.27,20.96
ELECTRICAL,5.57,-11.16,50.57
INSURANCE,4.54,4.52,29.13
PHARMA,3.41,-0.59,34.34
AUTO,3.36,-17.38,129.91


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3306542.0,20
XR,1262551.0,13
AR,1260290.0,10
X40,980849.0,14
OX40N,692257.0,10
X40N,685393.0,9
XY25,464325.0,7
SR,274180.0,2
MH,63524.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3460246.0,25
M-SC,1406505.0,16
X-LC,1196645.0,11
X-MC,1124954.0,15
X-SC,828356.0,8
H-MC,398764.0,3
L-SC,253773.0,3
H-LC,137568.0,2
M-LC,108766.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1153210.0      6
           AR         842556.0      5
           XR         789316.0      7
M-SC       XY24       737304.0      6
X-LC       XY24       515576.0      2
X-MC       X40        406599.0      7
X-LC       X40        406066.0      5
X-SC       XY24       375004.0      2
X-MC       XY24       344083.0      3
H-SC       OX40N      337460.0      4
X-SC       X40N       285168.0      4
H-SC       SR         274180.0      2
M-SC       OX40N      269029.0      5
H-MC       AR         217399.0      2
X-LC       X40N       202203.0      2
X-MC       X40N       198022.0      3
H-MC       XY24       181365.0      1
X-MC       XY25       176250.0      2
X-SC       X40        168184.0      2
L-SC       XR         168005.0      2
M-SC       XR         142310.0      2
           XY25       133747.0      1
           AR         124115.0      2
M-LC       XR         108766.0      1
L-SC       OX40N       85768.0      1
H-LC       AR          76220.0      1
X-LC       XY25        72800.0      2
H-SC       MH          63524.0      1
H-LC       X200        61348.0      1
L-MC       XR          54154.0      1
M-MC       XY25        45835.0      1
L-LC       XY25        35693.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 52.0 seconds
